In [28]:
import numpy as np
import numpy.linalg as la

In [3]:
points_left = [[958, 38, 1],
               [1117, 111, 1],
               [874, 285, 1],
               [707, 218, 1],
               [292, 569, 1],
               [770, 969, 1],
               [770, 1465, 1],
               [317, 1057, 1]]
points_right = [[933, 33, 1],
                [1027, 132, 1],
                [692, 223, 1],
                [595, 123, 1],
                [272, 360, 1],
                [432, 814, 1],
                [414, 1284, 1],
                [258, 818, 1]]

points_left = np.array(points_left)
points_right = np.array(points_right)

In [94]:
print('LEVA \t--> DESNA: ', '==================================', sep='\n')
print('\n'.join([f'{(a[0], a[1], a[2])} \t--> {(b[0], b[1], b[2])}' for a, b in zip(points_left, points_right)]))


LEVA 	--> DESNA: 
(958, 38, 1) 	--> (933, 33, 1)
(1117, 111, 1) 	--> (1027, 132, 1)
(874, 285, 1) 	--> (692, 223, 1)
(707, 218, 1) 	--> (595, 123, 1)
(292, 569, 1) 	--> (272, 360, 1)
(770, 969, 1) 	--> (432, 814, 1)
(770, 1465, 1) 	--> (414, 1284, 1)
(317, 1057, 1) 	--> (258, 818, 1)


In [95]:
def get_vec_product_matrix(v):
    p1, p2, p3 = v
    return np.array([[0, -p3, p2],
                     [p3, 0, -p1],
                     [-p2, p1, 0]])

In [24]:
def get_eq(x, y):
    a1, a2, a3 = x
    b1, b2, b3 = y
    return np.array([a1*b1, a2*b1, a3*b1, a1*b2, a2*b2, a3*b2, a1*b3, a2*b3, a3*b3])
    

In [29]:
def get_f_system(xs, ys):
    return np.array([get_eq(x, y) for x, y in zip(xs, ys)])

In [83]:
def solve_system_matrix(M):
    _, _, v = la.svd(M)
    return np.reshape(v[-1], (3, 3))


In [90]:
def get_epipoles(F):
    u, _, v = la.svd(F)
    e1 = v[-1]
    e2 = u.T[-1]
    e1 = (1.0/e1[2])*e1
    e2 = (1.0/e2[2])*e2
    return e1, e2

In [117]:
def get_T1():
    return np.hstack((np.eye(3), np.array([[0], [0], [0]])))

def get_T2():

    pass

In [111]:
?np.vstack

Signature: np.vstack(tup)
Docstring:
Stack arrays in sequence vertically (row wise).

This is equivalent to concatenation along the first axis after 1-D arrays
of shape `(N,)` have been reshaped to `(1,N)`. Rebuilds arrays divided by
`vsplit`.

This function makes most sense for arrays with up to 3 dimensions. For
instance, for pixel-data with a height (first axis), width (second axis),
and r/g/b channels (third axis). The functions `concatenate`, `stack` and
`block` provide more general stacking and concatenation operations.

Parameters
----------
tup : sequence of ndarrays
    The arrays must have the same shape along all but the first axis.
    1-D arrays must have the same length.

Returns
-------
stacked : ndarray
    The array formed by stacking the given arrays, will be at least 2-D.

See Also
--------
concatenate : Join a sequence of arrays along an existing axis.
stack : Join a sequence of arrays along a new axis.
block : Assemble an nd-array from nested lists of blocks.
hstac

In [116]:
np.eye(3)


array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [97]:
F = solve_system_matrix(get_f_system(points_left, points_right))
e1, e2 = get_epipoles(F)